In [18]:
import Pkg; Pkg.activate(@__DIR__); Pkg.instantiate()

  Activating environment at `/mnt/064AC6424AC62E6D/git_workspace/flyhopper/scripts/Julia/Project.toml`


In [19]:
using RigidBodyDynamics
using LinearAlgebra
using MeshCatMechanisms
using StaticArrays

In [20]:
ℓ1 = 1.0
ℓ2 = 1.0
ℓc1 = ℓ1/2
ℓc2 = ℓ2/2
m1 = 1.0
m2 = 1.0
g = 9.81
I1 = m1*(ℓ1^2)
I2 = m2*(ℓ2^2)

1.0

In [21]:
g = -9.81 # gravitational acceleration in z-direction
world = RigidBody{Float64}("world")
doublependulum = Mechanism(world; gravity = SVector(0, 0, g))

Spanning tree:
Vertex: world (root)
No non-tree joints.

In [36]:
# We'll now add a second body, called 'upper link', to the `Mechanism`. 
# We'll attach it to the world with a revolute joint, with the $y$-axis 
# as the axis of rotation. We'll start by creating a `SpatialInertia`, 
# which stores the inertial properties of the new body:

axis = SVector(0., 1., 0.) # joint axis
frame0 = CartesianFrame3D("frame0") # the reference frame in which the spatial inertia will be expressed
inertia1 = SpatialInertia(frame0,
    moment=I1 * axis * axis',
    com=SVector(0, 0, ℓc1),
    mass=m1)

link0 = RigidBody(inertia1)
joint0 = Joint("joint0", Revolute(axis))
before_joint0_to_world = one(Transform3D, frame_before(joint0), default_frame(world))

# Now we can attach link0 to the world:
attach!(doublependulum, world, link0, joint0, joint_pose = before_joint0_to_world)
# which changes the tree representation of the `Mechanism`.

inertia2 = SpatialInertia(CartesianFrame3D("frame1"),
    moment=I2 * axis * axis',
    com=SVector(0, 0, ℓc2),
    mass=m2)
link1 = RigidBody(inertia2)
joint1 = Joint("joint1", Revolute(axis))
before_joint1_to_after_joint0 = Transform3D(frame_before(joint1), frame_after(joint0), SVector(0, 0, ℓ1))
attach!(doublependulum, link0, link1, joint1, joint_pose = before_joint1_to_after_joint0)

Spanning tree:
Vertex: world (root)
  Vertex: frame0, Edge: joint0
  Vertex: frame0, Edge: joint0
    Vertex: lower_link, Edge: elbow
No non-tree joints.

In [23]:
state = MechanismState(doublependulum)

MechanismState{Float64, Float64, Float64, …}(…)

In [37]:
set_configuration!(state, link0, 0.3)
set_configuration!(state, link1, 0.4)
set_velocity!(state, link0, 1.)
set_velocity!(state, link1, 2.);

LoadError: MethodError: no method matching set_configuration!(::MechanismState{Float64, Float64, Float64, TypeSortedCollections.TypeSortedCollection{Tuple{Vector{Joint{Float64, Revolute{Float64}}}}, 1}}, ::RigidBody{Float64}, ::Float64)
[0mClosest candidates are:
[0m  set_configuration!(::MechanismState, [91m::Joint[39m, ::Any) at /home/ben/.julia/packages/RigidBodyDynamics/8B04X/src/mechanism_state.jl:397
[0m  set_configuration!([91m::MechanismVisualizer[39m, ::Any...) at /home/ben/.julia/packages/MeshCatMechanisms/QczZo/src/visualizer.jl:166
[0m  set_configuration!([91m::AbstractVector{T} where T[39m, [91m::Joint{var"#s6", var"#s5"} where {var"#s6", var"#s5"<:Prismatic}[39m, ::Number) at /home/ben/.julia/packages/RigidBodyDynamics/8B04X/src/joint_types/prismatic.jl:36
[0m  ...

In [25]:
q = configuration(state)
v = velocity(state)

1-element SegmentedVector{JointID, Float64, Base.OneTo{JointID}, Vector{Float64}}:
 0.0

In [26]:
#=
curdir = pwd()
urdfpath = joinpath(curdir, "../../res/flyhopper_mockup/urdf/flyhopper_mockup_jl.urdf")
bot = parse_urdf(urdfpath, floating=False)
# remove_fixed_tree_joints!(bot);
g = -9.81 # gravitational acceleration in z-direction
state = MechanismState(bot)
set_configuration!(state, [1; 0; 0; 0; 0; 0; 0.7])
q = configuration(state)
v = velocity(state)
ts, qs, vs = simulate(state, 5., Δt = 1e-3);
=#

In [27]:
function f(x)
    # double pendulum dynamics
    
    q1 = x[1]
    q1̇ = x[2]
    q2 = x[3]
    q2̇ = x[4]
    
    M = zeros(2, 2)
    M[1, 1] = I1 + I2 + m2*ℓ1^2 + 2*m2*ℓ1*ℓc2*cos(q2)
    M[1, 2] = I2 + m2*ℓ1*ℓc2*cos(q2)
    M[2, 1] = I2 + m2*ℓ1*ℓc2*cos(q2)
    M[2, 2] = I2
    
    C = zeros(2, 2)
    C[1, 1] = -2*m2*ℓ1*ℓc2*sin(q2)*q2̇ 
    C[1, 2] = -m2*ℓ1*ℓc2*sin(q2)*q2̇ 
    C[2, 1] = m2*ℓ1*ℓc2*sin(q2)*q1̇
    C[2, 2] = 0
    
    τg = zeros(2)
    τg[1] = -m1*g*ℓc1*sin(q1) - m2*g*(ℓ1*sin(q1) + ℓc2*sin(q1+q2))
    τg[2] = -m2*g*ℓc2*sin(q1+q2)
    q_d = zeros(2)
    q_d[1] = q1̇ 
    q_d[2] = q2̇ 
    q_dd = inv(M)*(τg - C*q_d)
    q1̈  = q_dd[1]
    q2̈  = q_dd[2]
    
    ẋ = zeros(4)
    ẋ[1] = q1̇  # q1 dot
    ẋ[2] = q1̈  # q1 double dot
    ẋ[3] = q2̇  # q2 dot
    ẋ[4] = q2̈  # q2 double dot
    
end

f (generic function with 1 method)

In [28]:
function euler_step(xk)
    xn = xk .+ h*f(xk)
end

euler_step (generic function with 1 method)

In [29]:
function simulate!(xtraj, N)
    for k = 1:(N-1)
        xtraj[:,k+1] .= euler_step(xtraj[:,k])
    end
end

simulate! (generic function with 1 method)

In [30]:
Tf = 10.0
h = 0.001 #20 Hz
N = Int(floor(Tf./h + 1))
thist = h.*Array(0:(N-1));

In [31]:
x0 = [5*(pi/180); 0.0; 0.0; 0.0]
xtraj = zeros(4,N)
xtraj[:,1] = x0;

In [32]:
simulate!(xtraj, N)

In [33]:
mvis = MechanismVisualizer(doublependulum, Skeleton(randomize_colors=true));
render(mvis)

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://127.0.0.1:8702
└ @ MeshCat /home/ben/.julia/packages/MeshCat/GlCMx/src/visualizer.jl:73


LoadError: Principal inertias [-0.25, 0.0, 0.75] do not satisfy the triangle inequalities, so the equivalent inertial ellipsoid is not well-defined.

In [35]:
# Now we can simply call `simulate`, which will return a tuple consisting of:
# * simulation times (a `Vector` of numbers)
# * joint configuration vectors (a `Vector` of `Vector`s)
# * joint velocity vectors (a `Vector` of `Vector`s)

# ts, qs, vs = simulate(state, final_time, simple_control!; Δt = 1e-3);
# And animate:
MeshCatMechanisms.animate(mvis, ts, qs; realtimerate = 1.);

LoadError: UndefVarError: mvis not defined